# Author: Kumar R.

# Project: Image Scrapper

In [1]:
#Load the required libraries
import os
import time
import requests
from selenium import webdriver

In [2]:
def fetch_image_urls(query: str, max_links_to_fetch: int, wd: webdriver, sleep_between_interactions: int = 1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

        # build the google query

    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)

        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [3]:
def persist_image(folder_path:str,url:str,counter):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb')
        f.write(image_content)
        f.close()
        print(f"SUCCESS - saved {url} - as {folder_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")
    

In [4]:
def search_and_download(search_term: str, driver_path: str, target_path='./images', number_images=10):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' '))) # make the folder name inside images with the search string

    if not os.path.exists(target_folder):
        os.makedirs(target_folder) # make directory using the target path if it doesn't exist already

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)

    counter = 0
    for elem in res:
        persist_image(target_folder, elem, counter)
        counter += 1


In [5]:
DRIVER_PATH = './chromedriver'
search_term = 'Autobahn'
# num of images you can pass it from here  by default it's 10 if you are not passing
# number_images = 10
search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

Found: 100 search results. Extracting links from 0:100
Found: 10 image links, done!
SUCCESS - saved https://www.autobahn.de/fileadmin/_processed_/7/0/csm_2020_12_19_NORDOST_AD_Havelland_A24_A10_Sunset_klar_02_80aecaf99a.jpg - as ./images\autobahn
SUCCESS - saved https://static01.nyt.com/images/2020/08/19/world/19germany-slamist01/merlin_175890681_02a3d176-6a5d-4e56-84af-70452e44e08c-mobileMasterAt3x.jpg - as ./images\autobahn
SUCCESS - saved https://bnn.de/karlsruhe/autobahn-4-yrx7ai/alternates/LANDSCAPE_13x7_BASE/autobahn-4 - as ./images\autobahn
SUCCESS - saved https://www.dvz.de/fileadmin/_processed_/b/6/csm_Autobahn-GmbH_iStock-140469_1093cd9854.jpg - as ./images\autobahn
SUCCESS - saved https://assets.adac.de/image/upload/c_scale,f_auto,t_1:1-default,w_1500/v1/ADAC-eV/KOR/Bilder/RF/news-autobahneroeffnung-2001_yjq6fn - as ./images\autobahn
SUCCESS - saved https://www.tagesschau.de/multimedia/bilder/autobahn-199~_v-videowebl.jpg - as ./images\autobahn
SUCCESS - saved https://www.fr